In [1]:
from datasets.stored_dataframe import CapgMyoDataModule, CslDataModule, NinaProDataModule
from models import Classifier, OriginalModel
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## First Experiment

In [2]:
pl.seed_everything(42, workers=True)
lr_lambda=lambda epoch: 1 if epoch<16 else 0.1 if epoch<24 else 0.01
model_capgmyo = OriginalModel(8, 16, 8)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR
classifier = Classifier(model_capgmyo, lr_scheduler=lr_scheduler, lr=0.01, weight_decay=0.0001, lr_lambda=lr_lambda)
logger_capgmyo = WandbLogger(project="EMG Armband", name="CapgMyo")
data_module_capgmyo = CapgMyoDataModule(
    batch_size=1000
)
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=3, mode='max')
trainer_capgmyo = pl.Trainer(gpus=-1, max_epochs=28, logger=logger_capgmyo, accelerator="gpu", callbacks=[early_stop_callback])

Global seed set to 42
wandb: Currently logged in as: grigori200. Use `wandb login --relogin` to force relogin


C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:95: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:114: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:133: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
trainer_capgmyo.fit(model=classifier, datamodule=data_module_capgmyo)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | OriginalModel    | 4.6 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.6 M     Trainable params
0         Non-trainable params
4.6 M     Total params
18.291    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:107: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:88: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [4]:
trainer_capgmyo.test(model=classifier, datamodule=data_module_capgmyo)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:126: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          loss              0.27983471751213074
   test_accuracy_macro      0.28371965885162354
   test_accuracy_micro      0.8966666460037231
      test_f1_macro         0.28310224413871765
      test_f1_micro         0.8966666460037231
  test_precision_macro      0.29033350944519043
  test_precision_micro      0.8966666460037231
 test_specificity_macro     0.9814516305923462
 test_specificity_micro     0.9849238395690918
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss': 0.27983471751213074,
  'test_accuracy_micro': 0.8966666460037231,
  'test_accuracy_macro': 0.28371965885162354,
  'test_f1_micro': 0.8966666460037231,
  'test_f1_macro': 0.28310224413871765,
  'test_precision_micro': 0.8966666460037231,
  'test_precision_macro': 0.29033350944519043,
  'test_specificity_micro': 0.9849238395690918,
  'test_specificity_macro': 0.9814516305923462}]

## Second Experiment

In [2]:
pl.seed_everything(42, workers=True)
model_ninapro = OriginalModel(52, 1, 10)
classifier = Classifier(model_ninapro, lr=0.001, weight_decay=0.0001)
logger_ninapro = WandbLogger(project="EMG Armband", name="NinaPro")
data_module_ninapro = NinaProDataModule(
    batch_size=1000
)
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=3, mode='max')
trainer_ninapro = pl.Trainer(gpus=-1, max_epochs=50, logger=logger_ninapro, accelerator="gpu", callbacks=[early_stop_callback])

Global seed set to 42
wandb: Currently logged in as: grigori200. Use `wandb login --relogin` to force relogin


C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:95: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:114: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:133: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer_ninapro.fit(model=classifier, datamodule=data_module_ninapro)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | OriginalModel    | 711 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
711 K     Trainable params
0         Non-trainable params
711 K     Total params
2.848     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\mateu\anaconda3\envs\emg\lib\site-packages\pytorch_lightning\core\datamodule.py:107: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


In [ ]:
trainer_ninapro.test(model=classifier, datamodule=data_module_ninapro)